## import packages

In [24]:
import os
from datetime import datetime, timedelta
import pandas as pd

from pandas import DataFrame, Series
from typing import List

from dotenv import load_dotenv

load_dotenv()

True

## Variables

### Env

In [25]:
INPUT_FOLDER: str = os.getenv('INPUT_FOLDER_PATH')

### other

In [26]:
time_columns: list = ['System', 'Queue', 'Ring', 'Talk', 'Hold', 'ACW', 'Consult', 'Disposition', 'Duration']
vdn_flow_columns: list = ['1', '2', '3', '4', '5', '6', '7', '8', '9']
call_work_codes_columns: list = ['1.1', '2.1', '3.1', '4.1', '5.1']

## Functions

### summarize_unused_columns

In [27]:
def summarize_unused_columns(dataset_column_index: pd.core.indexes.base.Index) -> List[str]:
   """
   
   """
   unused_column_names: List[str] = [col_name for col_name in dataset_column_index if 'unnamed' in str(col_name).lower()]
   return unused_column_names

### filter_unused_columns

In [28]:
def filter_unused_columns(dataset: DataFrame) -> DataFrame:
    """
    
    """
    input_dataset: DataFrame = dataset.copy()
    unused_column_names: List[str] = summarize_unused_columns(dataset.columns)
    clean_df: DataFrame = input_dataset.drop(labels=unused_column_names, axis='columns')
    return clean_df

### set_datetime_dtype_values

In [29]:
def set_datetime_dtype_values(dataset: DataFrame, columns: List[str]) -> DataFrame:
    """
    
    """
    for col_name in columns:
        dataset[col_name] = dataset[col_name].map(lambda str_value: datetime.strptime(str_value, "%Y-%m-%d %H:%M:%S"))

    return dataset

### timedelta_from_string

In [30]:
def timedelta_from_string(timestring: str) -> timedelta:
    """
    
    """
    time_obj = datetime.strptime(timestring,"%H:%M:%S")
    timedelta_obj: timedelta = timedelta(hours=time_obj.hour, minutes=time_obj.minute, seconds=time_obj.second)
    return timedelta_obj

### timestring_to_seconds

In [31]:
def timestring_to_seconds(dataset: DataFrame, columns: List[str]) -> DataFrame:
    """
    
    """
    for col_name in columns:
        dataset[col_name] = dataset[col_name].map(lambda timestring: timedelta_from_string(timestring))

    return dataset

### get_avg_time

In [32]:
def get_avg_time(dataset: DataFrame, columns: List[str]) -> timedelta:
    """
    
    """
    subset: DataFrame = dataset[columns].copy()
    subset['row_sum'] = subset.sum(axis='columns')
    average: timedelta = subset['row_sum'].mean()
    return average

## core logic

### test read

In [33]:
test_df: DataFrame = pd.read_excel(os.path.join(INPUT_FOLDER, 'Week 1 -Mei.xlsx'), header=1)
test_df.head()

,Unnamed: 0,CallId,Segment,Start,Stop,Calling Party,Trunk Group,Dialed Number,Answered Agent,Original Agent,...,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,1.1,2.1,3.1,4.1,5.1
0,NaN,582539,1,2025-05-06 08:00:33,2025-05-06 08:12:07,2807 (Astrid Nederlof),NaN,8008 (SGD Serviceloket),95120 (Laurens Brand),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,582524,1,2025-05-06 08:01:02,2025-05-06 08:05:02,0620485126,101 (Vodafone01),8008 (SGD Serviceloket),95121 (Jordy Cruz),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,582547,1,2025-05-06 08:04:04,2025-05-06 08:16:12,7114 (Prya Bhola),NaN,8008 (SGD Serviceloket),95022 (Yamani Mondt),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,582561,1,2025-05-06 08:04:46,2025-05-06 08:22:13,NaN,NaN,8008 (SGD Serviceloket),95121 (Jordy Cruz),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,582544,1,2025-05-06 08:05:10,2025-05-06 08:14:30,0639621473,101 (Vodafone01),8008 (SGD Serviceloket),95120 (Laurens Brand),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### test filter unused columns

In [34]:
test_df = filter_unused_columns(dataset=test_df)
test_df.columns

Index([          'CallId',          'Segment',            'Start',
                   'Stop',    'Calling Party',      'Trunk Group',
          'Dialed Number',   'Answered Agent',   'Original Agent',
       'Call Disposition',           'Vector',         'Priority',
                  'Split',            'Level',           'System',
                  'Queue',             'Ring',             'Talk',
                   'Hold',              'ACW',          'Consult',
            'Disposition',         'Duration',                  1,
                        2,                  3,                  4,
                        5,                  6,                  7,
                        8,                  9,              '1.1',
                    '2.1',              '3.1',              '4.1',
                    '5.1'],
      dtype='object')

### test set datetime dtype

In [35]:
test_value = test_df["Start"][0]
print(test_value, type(test_value))

2025-05-06 08:00:33 <class 'str'>


In [36]:
test_df = set_datetime_dtype_values(test_df, columns=['Start'])

In [37]:
test_value = test_df["Start"][0]
print(test_value, type(test_value))

2025-05-06 08:00:33 <class 'pandas._libs.tslibs.timestamps.Timestamp'>


### test timestring to seconds

In [38]:
test_value = test_df['System'][0]
print(test_value, type(test_value))

00:00:59 <class 'str'>


In [39]:
result = timedelta_from_string(test_value)
result

datetime.timedelta(seconds=59)

In [40]:
test_df = timestring_to_seconds(test_df, ['System'])

In [41]:
test_value = test_df['System'][0]
print(test_value, type(test_value))

0 days 00:00:59 <class 'pandas._libs.tslibs.timedeltas.Timedelta'>


### test avg calculations

In [42]:
test_df = timestring_to_seconds(test_df, time_columns)
avg_reaction_time: timedelta = get_avg_time(test_df, columns=['System', 'Queue', 'Ring'])
avg_reaction_time

TypeError: strptime() argument 1 must be str, not Timedelta